# CNN for AdvSND target energy reconstruction

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import hist
import matplotlib.pyplot as plt

plt.style.use(["science", "notebook"])

In [ ]:
plt.rcParams["font.size"] = 14
plt.rcParams["axes.formatter.limits"] = -5, 4
plt.rcParams["figure.figsize"] = 6, 4
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
import plotting

In [ ]:
df = pd.read_csv("features_new.csv")

In [ ]:
hitmaps = np.load("images_1000.npy")

In [ ]:
nu_energy_dep_target = df.pop("energy_dep_target").values

nu_energy_dep_target = nu_energy_dep_target[:1000]

In [ ]:
start_z = df.pop("start_z").values

start_z = start_z[:1000]

In [ ]:
y = start_z

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
hitmaps_v = hitmaps[:, :, ::2]
hitmaps_h = hitmaps[:, :, 1::2]

In [ ]:
hitmaps_v_T = np.swapaxes(hitmaps_v, 1, 2)
hitmaps_h_T = np.swapaxes(hitmaps_h, 1, 2)

In [ ]:
target_X_v = np.expand_dims(hitmaps_v_T, 3)
target_X_h = np.expand_dims(hitmaps_h_T, 3)

In [ ]:
X_h_train, X_h_test, X_v_train, X_v_test, y_train, y_test = train_test_split(
    target_X_h, target_X_v, y, random_state=0
)

In [ ]:
event = 7
fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
fig.suptitle("CNN input image")
fig.subplots_adjust(wspace=0)
ax1.imshow(hitmaps_h[event], aspect=0.05)
ax1.set_ylabel("Channel")

ax2.imshow(hitmaps_v[event], aspect=0.05)

ax1.set_xlabel("Station")
ax2.set_xlabel("Station")
fig.savefig(f"plots/CNN_input_event_{event}.pdf")
fig.savefig(f"plots/CNN_input_event_{event}.png")

In [ ]:
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Concatenate
import tensorflow.keras.optimizers
import tensorflow.keras.metrics
import tensorflow.keras.losses
from tensorflow.keras.models import Model

import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")

In [ ]:
drop_middle = 0.5
target_shape = (100, 3072, 1)

lr = 2e-4  # ; betaa1=30; betaa2=100; decay=1e-3

# AdvTarget

target_h_input = Input(target_shape, name="target_h_in")
X_h = Conv2D(
    16, kernel_size=(3, 4), padding="same", activation="elu", name="target_h_conv1_1"
)(target_h_input)
X_h = Conv2D(
    16, kernel_size=(3, 4), padding="same", activation="elu", name="target_h_conv1_2"
)(X_h)
X_h = MaxPooling2D(pool_size=(1, 4), padding="valid", name="target_h_pool1")(X_h)

X_h = Conv2D(
    32, kernel_size=(3, 4), padding="same", activation="elu", name="target_h_conv2_1"
)(X_h)
X_h = MaxPooling2D(pool_size=(1, 4), padding="valid", name="target_h_pool2")(X_h)

X_h = Dropout(rate=drop_middle, name="target_h_drop_middle")(X_h)

X_h = Conv2D(
    32, kernel_size=(3, 4), padding="same", activation="elu", name="target_h_conv3_1"
)(X_h)
X_h = MaxPooling2D(pool_size=(1, 2), padding="valid", name="target_h_pool3")(X_h)

X_h = Conv2D(
    64, kernel_size=(3, 3), padding="same", activation="elu", name="target_h_conv4_1"
)(X_h)
X_h = Conv2D(
    64, kernel_size=(3, 3), padding="same", activation="elu", name="target_h_conv4_2"
)(X_h)
X_h = MaxPooling2D(pool_size=(2, 2), padding="same", name="target_h_pool4")(X_h)
X_h = Flatten()(X_h)

target_v_input = Input(target_shape, name="target_v_in")
X_v = Conv2D(
    16, kernel_size=(3, 4), padding="same", activation="elu", name="target_v_conv1_1"
)(target_v_input)
X_v = Conv2D(
    16, kernel_size=(3, 4), padding="same", activation="elu", name="target_v_conv1_2"
)(X_v)
X_v = MaxPooling2D(pool_size=(1, 4), padding="valid", name="target_v_pool1")(X_v)

X_v = Conv2D(
    32, kernel_size=(3, 4), padding="same", activation="elu", name="target_v_conv2_1"
)(X_v)
X_v = MaxPooling2D(pool_size=(1, 4), padding="valid", name="target_v_pool2")(X_v)

X_v = Dropout(rate=drop_middle, name="target_v_drop_middle")(X_v)

X_v = Conv2D(
    32, kernel_size=(3, 4), padding="same", activation="elu", name="target_v_conv3_1"
)(X_v)
X_v = MaxPooling2D(pool_size=(1, 2), padding="valid", name="target_v_pool3")(X_v)

X_v = Conv2D(
    64, kernel_size=(3, 3), padding="same", activation="elu", name="target_v_conv4_1"
)(X_v)
X_v = Conv2D(
    64, kernel_size=(3, 3), padding="same", activation="elu", name="target_v_conv4_2"
)(X_v)
X_v = MaxPooling2D(pool_size=(2, 2), padding="same", name="target_v_pool4")(X_v)
X_v = Flatten()(X_v)

X = Concatenate()([X_h, X_v])
X_cl = Dense(1)(X)

toy_model = Model(
    inputs=[target_h_input, target_v_input], outputs=X_cl, name="full_toy_model"
)

K.clear_session()

adamka = tf.keras.optimizers.Adam(learning_rate=lr)
toy_model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

In [ ]:
toy_model.summary()

In [ ]:
fit_result = toy_model.fit(
    x=[X_h_train, X_v_train], y=y_train, batch_size=128, epochs=30
)

In [ ]:
retoy_model = tf.keras.models.load_model("toy_start_z_1000.keras")

In [ ]:
toy_model.save("toy_start_z_1000.keras")

In [ ]:
history_df = pd.DataFrame(fit_result.history)
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
plt.title("CNN start z")
history_df.loss.plot(ax=ax1, color=colors[0])
ax1.set_xlabel("Epochs")
ax1.set_ylabel("Loss Function", color=colors[0])
history_df.mae.plot(ax=ax2, color=colors[1])
ax2.set_ylabel("Error", color=colors[1])
plotting.watermark("")
plt.savefig("plots/CNN_start_z_epochs.png")
plt.savefig("plots/CNN_start_z_epochs.pdf")

In [ ]:
# test=retoy_model.predict(x=[x_test['scifi_h'], x_test['scifi_v'], x_test['us'], x_test['ds']])
y_pred = toy_model.predict(x=[X_h_test, X_v_test])

In [ ]:
rms = tensorflow.keras.metrics.RootMeanSquaredError()

rms.update_state(y_test, y_pred)

rms.result()

In [ ]:
h = hist.Hist.new.Regular(10, -30, +120, name=r"𝛥z [cm]").Double()

In [ ]:
h.fill(np.squeeze(y_pred) - np.squeeze(y_test))

In [ ]:
import scipy

In [ ]:
def model(x, mu, sigma):
    return scipy.stats.norm.cdf(x, mu, sigma)

In [ ]:
entries, edges = h.to_numpy()

In [ ]:
from iminuit import cost
from iminuit import Minuit

In [ ]:
m = Minuit(cost.BinnedNLL(entries, edges, model), 0, 25)

In [ ]:
res = m.migrad()

In [ ]:
h.plot()
plt.xlabel(r"$\Delta z\;[\mathrm{cm}]$")
plot_range = edges[0], edges[-1]
x = np.linspace(*plot_range, 100)
best_fit = scipy.stats.norm(res.params[0].value, res.params[1].value)
# best_fit = scipy.stats.norm(0.044, 2.83) # TODO take from fit
n_bins = 10
binsize = (plot_range[1] - plot_range[0]) / n_bins
scale = h.sum() / (best_fit.cdf(plot_range[1]) - best_fit.cdf(plot_range[0])) * binsize
plt.plot(x, scale * best_fit.pdf(x))
ax = plt.gca()
# plt.text(0.6, 0.9, r"$\mu = 0.044 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.9,
    rf"$\mu = {res.params[0].value:.2f} \pm {res.params[0].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
# plt.text(0.6, 0.81, r"$\sigma = 2.83 $\;cm", transform=ax.transAxes, usetex=True)
plt.text(
    0.6,
    0.81,
    rf"$\sigma = {res.params[1].value:.2f} \pm {res.params[1].error:.2f}$\;cm",
    transform=ax.transAxes,
    usetex=True,
)
plotting.watermark()
plt.savefig("plots/h_dz_CNN.pdf")
plt.savefig("plots/h_dz_CNN.png")